# Training and Testing A VGG Model
Here we test a VGG from reminder change link!!! [Very Deep Convolutional Networks for Large-Scale Image Recognition](https://arxiv.org/abs/1409.1556)

In [1]:
import torch
import torchvision as tv
import torchvision.transforms.v2 as v2
from our_datasets import Country_images
from DenseNet import DenseNet
#from Country_dict import comp_country_dict
import os

USE_GPU = True
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
#get models
print(f"Using {device} device")

Using cuda device


## Load our Dataset and Create our dataloaders

In [2]:
batch_size = 64
weights = tv.models.VGG19_BN_Weights.DEFAULT
transform = v2.Compose([weights.transforms(), ])


dataset_path = os.path.join("data","compressed_dataset")
dataset = Country_images("country.csv",dataset_path,transform=transform)
num_classes = dataset.get_num_classes()
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset,lengths=[0.7,0.1,0.2])
if device == 'cuda':
    data_loader_params = {
        'batch_size': batch_size,  # Batch size for data loading
        'num_workers': 8,  # Number of subprocesses to use for data loading
        'persistent_workers': True,  # If True, the data loader will not shutdown the worker processes after a dataset has been consumed once. This allows to maintain the worker dataset instances alive.
        'pin_memory': True,  # If True, the data loader will copy Tensors into CUDA pinned memory before returning them. Useful when using GPU.
        'pin_memory_device': 'cuda' ,  # Specifies the device where the data should be loaded. Commonly set to use the GPU.
    }
else:
    data_loader_params = {
        'batch_size': batch_size,  # Batch size for data loading
        #'num_workers': 8,  # Number of subprocesses to use for data loading
        #'persistent_workers': True,  # If True, the data loader will not shutdown the worker processes after a dataset has been consumed once. This allows to maintain the worker dataset instances alive.
        #'pin_memory': True,  # If True, the data loader will copy Tensors into CUDA pinned memory before returning them. Useful when using GPU.
        #'pin_memory_device': 'cuda' ,  # Specifies the device where the data should be loaded. Commonly set to use the GPU.
    }
train_dataloader      = torch.utils.data.DataLoader(train_dataset, **data_loader_params, shuffle=True)
val_dataloader        = torch.utils.data.DataLoader(val_dataset, **data_loader_params, shuffle=True)
test_dataloader       = torch.utils.data.DataLoader(test_dataset, **data_loader_params, shuffle=False)

## Load our model

In [ ]:
#hyperparameters:
lr = 0.001
#maximum learning rate we will let our model train in order to train faster at the start
max_lr = 0.01
weight_decay = 0.000001
EPOCHS = 40
#end hyperparameters

#model and optimizers
model = tv.models.vgg19_bn(num_classes=num_classes)
#added data to our model for ease of use (and to prevent passing so many variables to our training function)
model.device = device
model.name = "VGG-19BN-1"
model.path = os.path.join("Trained_Models","CNN") #where to save our best model
print(model.path)

print(model)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
#scales the gradients, neccessary for mixed precision data types to properly converge
if device == 'cuda':
    scaler = torch.amp.GradScaler()
else:
    scaler = None
#change our learning rate based on far we are in training and if we are improving
lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=max_lr, total_steps=EPOCHS*len(train_dataloader))

Trained_Models\CNN
VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (1

## Train our Model

In [ ]:
#we will call the function we defined in "Training_Functions.py"
from Training_Functions import TrainModel
#Whether we want to train the model, else we test it
model_train = True
#Whether we want to load a model and fine tune it
fine_tune = True
#Whether we want to use Automatatic Mixed Precision(Pytorch needs to be new enough)
use_amp= True

if model_train and fine_tune:
    print("Starting Fine Tuning")
    checkpoint = torch.load(os.path.join(model.path,"DenseNet-201-Full-2"+"-Best.pth"),map_location=torch.device(device))
    model.load_state_dict(checkpoint)
    model = model.to(device)
    TrainModel(model,EPOCHS, loss_fn, train_dataloader, val_dataloader, optimizer, lr_scheduler, use_amp, scaler)
elif model_train:
    print("Starting Training")
    model = model.to(device)
    TrainModel(model,EPOCHS, loss_fn, train_dataloader, val_dataloader, optimizer, lr_scheduler, use_amp, scaler)
else:
    print("Loading Model")
    checkpoint = torch.load(os.path.join( model.path,model.name+"-Best.pth"),map_location=torch.device(device))
    model.load_state_dict(checkpoint)
    model = model.to(device)

Starting Training
EPOCH 1:
  batch 100 loss: 4.163289546966553
  batch 200 loss: 3.7160258293151855
  batch 300 loss: 3.5582242012023926
  batch 400 loss: 3.4835238456726074
  batch 500 loss: 3.4369733333587646
LOSS train 3.4369733333587646 valid 3.552894353866577
Accuracy: 0.21587830798082933
EPOCH 2:
  batch 100 loss: 3.2932915687561035
  batch 200 loss: 3.275733709335327
  batch 300 loss: 3.2578015327453613
  batch 400 loss: 3.252349615097046
  batch 500 loss: 3.2402429580688477
LOSS train 3.2402429580688477 valid 3.167269229888916
Accuracy: 0.25651177328610125
EPOCH 3:
  batch 100 loss: 3.2288386821746826
  batch 200 loss: 3.2197718620300293
  batch 300 loss: 3.2163772583007812
  batch 400 loss: 3.20436429977417
  batch 500 loss: 3.2020270824432373
LOSS train 3.2020270824432373 valid 3.164241313934326
Accuracy: 0.25651177328610125
EPOCH 4:
  batch 100 loss: 3.2599616050720215
  batch 200 loss: 3.2279086112976074
  batch 300 loss: 3.2269511222839355
  batch 400 loss: 3.2146937847137

## Test our Model

In [5]:
#we will call the function we defined in "Training_Functions.py"
from Training_Functions import TestModel
TestModel(model, test_dataloader, loss_fn)

Got 2423 / 9598 correct (25.24)


0.25244842675557405